In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = "{:,.2f}".format

from experiments import *

import sys

sys.path.insert(1, '../fuzzylearn/')

from fuzzylearn import *
from fuzzylearn.fuzzifiers import LinearFuzzifier, CrispFuzzifier,ExponentialFuzzifier,QuantileLinearPiecewiseFuzzifier, QuantileConstantPiecewiseFuzzifier
from fuzzylearn.kernel import GaussianKernel, LinearKernel, HyperbolicKernel, PolynomialKernel, HomogeneousPolynomialKernel
from fuzzylearn import solve_optimization_gurobi

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, RobustScaler, PowerTransformer, Normalizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score

dataset = pd.read_excel("dataset/IncidentiModificato.xlsx")
dataset = dataset.set_index("VERBALE")

In [2]:
import datetime as dt

dataset.DATA = dataset.DATA.apply(lambda d: (d - dt.datetime(1970,1,1)).days)

dataset.head()

,DATA,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,10893,0,81,84.00,1.75,27.43,0,1,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,10970,1,69,69.00,1.62,26.29,1,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,11026,1,71,67.00,1.55,27.89,1,2,0,1,...,0,0,0,0,4,6,0,0,4,10
86220,11122,1,54,60.00,1.59,23.73,1,4,0,0,...,0,0,0,0,4,5,3,2,4,14
86247,11130,1,78,69.00,1.67,24.74,1,2,0,0,...,0,0,0,0,4,2,0,2,4,8


In [3]:
import logging

# create logger
f_logger = logging.getLogger(__name__)
f_logger.setLevel(logging.INFO)

# create console handler and set level to debug
file = logging.FileHandler('esperimenti.log')
file.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s -%(message)s')

# add formatter to ch
file.setFormatter(formatter)

# add ch to logger
f_logger.addHandler(file)

# Defuzzification

In [3]:
from experiments import *

In [11]:
risultati_gaussian = pd.read_json("json_result/first_experiments20.json")
risultati_other_kernels = pd.read_json("json_result/experiments_otherkernels20.json")
risultati0 = pd.read_json("json_result/first_experiments20_class0.json")
risultati = risultati_gaussian.join(risultati_other_kernels)

In [18]:
scalers_table = pd.read_json("json_result/experiments_20feature_scalers.json")

In [12]:
def classify(item, est_mu,classes):
    return sorted([(l, est_mu[l](item)) for l in classes], key=lambda i:i[1], reverse=True)[0][0]

In [13]:
def to_membership_values(labels, target):
    return [1 if l==target else 0 for l in labels]

In [14]:
def best_estimator_holdout(best_estimators,data_index,data_values, data_labels,classes,train_percentage,classifier, num_holdouts):
    n = len(data_values)
    
    performance_train = performance_test = 0.0
    
    trials = range(num_holdouts)
    
    for trial in trials:
    
        permuted_indices = np.random.permutation(n)
        train_indices = permuted_indices[:int(n*train_percentage)]
        test_indices = permuted_indices[int(n*train_percentage):]
    
        train_values = [data_values[i] for i in train_indices]
        test_values = [data_values[i] for i in test_indices]
        
        train_values_arr = np.asarray(train_values)
        
        mu = {} 
        mu_train = {}
        mu_test = {}
        
        result = []
        
        for target in classes:
            mu[target] = to_membership_values(data_labels, target)
            mu_train[target] = [mu[target][i] for i in train_indices]
            mu_test[target] = [mu[target][i] for i in test_indices]
        
        labels_train = [data_labels[i] for i in train_indices]
        labels_test = [data_labels[i] for i in test_indices]
        
        est_mu = {}
        
        targets = classes
        
        for target,be in zip(targets,best_estimators):
            be.fit(train_values_arr, mu_train[target], n_jobs=-1)
            est_mu[target] = be.estimated_membership_

            if len(est_mu.values()) < len(classes):
                continue # at least one class not inferred
        
        index_train = [data_index[i] for i in train_indices]
        index_test = [data_index[i] for i in test_indices]
        
        results_train = list(zip(index_train,
                            map(lambda item: classifier(item, est_mu,classes),
                            train_values), labels_train))
        performance_train += 1.0 * len([r for r in results_train if r[1] != r[2]]) / len(results_train)
    
        results_test = list(zip(index_test,
                           map(lambda item: classifier(item, est_mu,classes),
                           test_values), labels_test))
    
        performance_test += 1.0 * len([r for r in results_test if r[1] != r[2]]) / len(results_test)
        
    return (performance_train/num_holdouts, performance_test/num_holdouts)

In [15]:
risultati

,Totale,Totali distretti,Totali,Lesioni 5 componenti,Lesioni 10 componenti,Lesioni 15 componenti,Lesioni 20 componenti,LesioniTotali5,LesioniTotali10,LesioniTotali15,...,All20,All25,All30,TotaleK,TotaliK,TotaliDistrettiK,LesioniTotali20K,Lesioni20K,All20K,BMI_Lesioni_Totali20K
c,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,...,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846
fuzzifier,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.QuantileConstant...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.QuantileConstant...,...,<class 'fuzzylearn.fuzzifiers.QuantileConstant...,<class 'fuzzylearn.fuzzifiers.QuantileConstant...,<class 'fuzzylearn.fuzzifiers.QuantileConstant...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>
k,GaussianKernel(0.1),GaussianKernel(0.7),GaussianKernel(0.7),GaussianKernel(0.9),GaussianKernel(0.9),GaussianKernel(0.5),GaussianKernel(0.3),GaussianKernel(0.9),GaussianKernel(0.9),GaussianKernel(0.3),...,GaussianKernel(0.4),GaussianKernel(0.4),GaussianKernel(0.4),HomogeneousPolynomialKernel(4),"HyperbolicKernel(1, 1)","HyperbolicKernel(1, 1)","HyperbolicKernel(1, 0)","HyperbolicKernel(1, 0)","HyperbolicKernel(1, 0)","HyperbolicKernel(1, 0)"
score,-0.31,-0.30,-0.30,-0.31,-0.47,-0.46,-0.32,-0.33,-0.33,-0.38,...,-0.32,-0.33,-0.33,-0.31,-0.31,-0.32,-0.29,-0.29,-0.29,-0.29


In [16]:
risultati0

,BMI_Lesioni_Totali5_0,BMI_Lesioni_Totali10_0,BMI_Lesioni_Totali15_0,BMI_Lesioni_Totali20_0,Anagrafica_Totale_0,Lesioni5_0,Lesioni10_0,Lesioni15_0,Lesioni20_0,Totale_0,TotaliDistretti_0,Totali_0,All5_0,All10_0,All15_0,All20_0,LesioniTotali5_0,LesioniTotali10_0,LesioniTotali15_0,LesioniTotali20_0
c,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846,0.021544346900318846
fuzzifier,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>
k,"HyperbolicKernel(1, 2)","HyperbolicKernel(1, 2)",LinearKernel(),LinearKernel(),"HyperbolicKernel(1, 0)","HyperbolicKernel(1, 1)",LinearKernel(),LinearKernel(),LinearKernel(),GaussianKernel(0.9),"HyperbolicKernel(1, 1)","HyperbolicKernel(1, 1)","HyperbolicKernel(1, 2)","HyperbolicKernel(1, 2)",LinearKernel(),LinearKernel(),"HyperbolicKernel(1, 1)","HyperbolicKernel(1, 2)",LinearKernel(),LinearKernel()
score,-0.25,-0.26,-0.26,-0.25,-0.28,-0.25,-0.25,-0.25,-0.24,-0.27,-0.26,-0.25,-0.25,-0.26,-0.26,-0.25,-0.25,-0.26,-0.25,-0.25


In [19]:
best_exp=[col for col in risultati.columns if risultati[col]['score'] == risultati.loc['score'].max()]
print(risultati[best_exp])
scalers_table[best_exp]

                                                  Lesioni20K
c                                       0.021544346900318846
fuzzifier  <class 'fuzzylearn.fuzzifiers.ExponentialFuzzi...
k                                     HyperbolicKernel(1, 0)
score                                                  -0.29


,Lesioni20K
"StandardScaler(copy=True, with_mean=True, with_std=True)",-0.29
"MinMaxScaler(copy=True, feature_range=(0, 1))",-0.32
MaxAbsScaler(copy=True),-0.32
"QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,\n output_distribution='normal', random_state=None,\n subsample=100000)",-0.28
"QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,\n output_distribution='uniform', random_state=None,\n subsample=100000)",-0.29
"Normalizer(copy=True, norm='l2')",-0.54
"RobustScaler(copy=True, quantile_range=(25, 75), with_centering=True,\n with_scaling=True)",-0.29
"PowerTransformer(copy=True, method='yeo-johnson', standardize=True)",-0.28


In [20]:
best_exp0=[col for col in risultati0.columns if risultati0[col]['score'] == risultati0.loc['score'].max()]
print(risultati0[best_exp0])
scalers_table[best_exp0]

                                               Lesioni20_0
c                                     0.021544346900318846
fuzzifier  <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>
k                                           LinearKernel()
score                                                -0.24


,Lesioni20_0
"StandardScaler(copy=True, with_mean=True, with_std=True)",-0.24
"MinMaxScaler(copy=True, feature_range=(0, 1))",-0.24
MaxAbsScaler(copy=True),-0.24
"QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,\n output_distribution='normal', random_state=None,\n subsample=100000)",-0.24
"QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,\n output_distribution='uniform', random_state=None,\n subsample=100000)",-0.25
"Normalizer(copy=True, norm='l2')",-0.34
"RobustScaler(copy=True, quantile_range=(25, 75), with_centering=True,\n with_scaling=True)",-0.24
"PowerTransformer(copy=True, method='yeo-johnson', standardize=True)",-0.26


In [21]:
dim_red = PCA(n_components=20)
scaling = QuantileTransformer(output_distribution='normal')

In [146]:
values = dataset[columns_lesioni].values
values_std = scaling.fit_transform(values)
values_20d = dim_red.fit_transform(values_std)

/home/manueldileo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (130). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [147]:
classes = (1,0)

In [153]:
be1 = FuzzyInductor(fuzzifier=ExponentialFuzzifier,c=0.021544346900318846,k=HyperbolicKernel(1,0))
be0 = FuzzyInductor(fuzzifier=LinearFuzzifier,c=0.021544346900318846,k=LinearKernel())
bes = [be1,be0]

In [147]:
results_df = pd.DataFrame()

In [148]:
perf_train, perf_test =best_estimator_holdout(bes,dataset.index,values_20d, dataset_labels,
                                              classes,0.7,classify,3)

100%|██████████| 100/100 [00:22<00:00,  4.80it/s]


In [149]:
results_df = results_df.append(dict(dict({'esperimento': best_exp[0][:-1],
                                'train_err': perf_train,
                                'test_err': perf_test},**{(k+"1"):v for k,v in be1.get_params().items() if k in ['k','fuzzifier','c']}),
                                   **{(k+"0"):v for k,v in be0.get_params().items() if k in ['k','fuzzifier','c']}), 
                                ignore_index=True)

In [150]:
results_df[['train_err','test_err']]

,train_err,test_err
0,0.30,0.26


#### Alpha-cut

In [155]:
from defuzzification import *
from functools import partial

In [156]:
"""
for alpha in [.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8]:
    perf_train, perf_test = best_estimator_holdout(bes,dataset.index,values_20d, dataset_labels,
                                              classes,0.7,partial(alpha_cut,alpha,0),3)
    print("alpha-cut {} : {} , {}".format(alpha,perf_train,perf_test))
"""

100%|██████████| 100/100 [00:49<00:00,  2.02it/s]


alpha-cut 0.2 : 0.3406593406593406 , 0.37606837606837606


100%|██████████| 100/100 [00:51<00:00,  1.93it/s]


alpha-cut 0.25 : 0.34432234432234426 , 0.35042735042735046


100%|██████████| 100/100 [00:51<00:00,  1.95it/s]


...

alpha-cut 0.7 : 0.3626373626373627 , 0.35042735042735035


100%|██████████| 100/100 [00:48<00:00,  2.08it/s]


alpha-cut 0.75 : 0.40659340659340665 , 0.4188034188034188


100%|██████████| 100/100 [00:50<00:00,  1.97it/s]


alpha-cut 0.8 : 0.4761904761904762 , 0.4102564102564103


In [157]:
"""
for alpha in [.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8]:
    perf_train, perf_test = best_estimator_holdout(bes,dataset.index,values_20d, dataset_labels,
                                              classes,0.7,partial(alpha_cut,alpha,1),3)
    print("alpha-cut {} : {} , {}".format(alpha,perf_train,perf_test))
"""

  0%|          | 0/100 [00:00<?, ?it/s]

alpha-cut 0.2 : 0.4945054945054945 , 0.641025641025641


  0%|          | 0/100 [00:00<?, ?it/s]

alpha-cut 0.25 : 0.5384615384615384 , 0.42735042735042733


  0%|          | 0/100 [00:00<?, ?it/s]

...

alpha-cut 0.7 : 0.45054945054945056 , 0.4102564102564103


  0%|          | 0/100 [00:00<?, ?it/s]

alpha-cut 0.75 : 0.4102564102564103 , 0.49572649572649574


100%|██████████| 100/100 [00:20<00:00,  4.90it/s]


alpha-cut 0.8 : 0.4468864468864469 , 0.48717948717948717


#### Parametro dell'ExponentialFuzzifier

In [152]:
class ExponentialFuzzifierFactory():
    def __init__(self): pass
            
    def exponential_fuzzifier_with_alpha(self,alpha):
        
        class ExponentialFuzzifierAlpha(ExponentialFuzzifier):
            def __init__(self,xs=None,mus=None):
                super().__init__(xs, mus,profile='alpha',alpha=alpha)
                
        return ExponentialFuzzifierAlpha

In [153]:
be0 = FuzzyInductor(fuzzifier=LinearFuzzifier,c=0.021544346900318846,k=LinearKernel())
results_ef = pd.DataFrame()

In [154]:
alpha = np.arange(0,1.1,.1)
for a in alpha:
    efa = ExponentialFuzzifierFactory().exponential_fuzzifier_with_alpha(a)
    be1 = FuzzyInductor(fuzzifier=efa,c=0.021544346900318846,k=HyperbolicKernel(1,0))
    bes = [be1,be0]
    perf_train, perf_test = best_estimator_holdout(bes,dataset.index,values_20d, dataset_labels,
                                              classes,0.7,classify,5)
    results_ef = results_ef.append({'alpha': a,
                                   'train': perf_train,
                                   'test': perf_test}, 
                                  ignore_index=True)

100%|██████████| 100/100 [00:25<00:00,  3.90it/s]
../fuzzylearn/fuzzylearn/fuzzifiers.py:254: RuntimeWarning: divide by zero encountered in log
  else np.exp(np.log(self.alpha)/q * (r - ssd))
100%|██████████| 100/100 [00:23<00:00,  4.28it/s]
../fuzzylearn/fuzzylearn/fuzzifiers.py:254: RuntimeWarning: divide by zero encountered in log
  else np.exp(np.log(self.alpha)/q * (r - ssd))
100%|██████████| 100/100 [00:23<00:00,  4.32it/s]
../fuzzylearn/fuzzylearn/fuzzifiers.py:254: RuntimeWarning: divide by zero encountered in log
  else np.exp(np.log(self.alpha)/q * (r - ssd))
100%|██████████| 100/100 [00:23<00:00,  4.18it/s]
../fuzzylearn/fuzzylearn/fuzzifiers.py:254: RuntimeWarning: divide by zero encountered in log
  else np.exp(np.log(self.alpha)/q * (r - ssd))
100%|██████████| 100/100 [00:23<00:00,  4.18it/s]
../fuzzylearn/fuzzylearn/fuzzifiers.py:254: RuntimeWarning: divide by zero encountered in log
  else np.exp(np.log(self.alpha)/q * (r - ssd))
100%|██████████| 100/100 [00:23<00:00,  

In [157]:
results_ef

,alpha,test,train
0,0.00,0.39,0.35
1,0.10,0.42,0.33
2,0.20,0.38,0.36
3,0.30,0.39,0.33
4,0.40,0.31,0.33
5,0.50,0.34,0.33
6,0.60,0.33,0.37
7,0.70,0.48,0.37
8,0.80,0.44,0.40
9,0.90,0.58,0.52
